In [ ]:
"""
Code for estimating population exposed to degraded land.
"""
# Copyright 2021 Conservation International

import ee
import json

# The service account email address authorized by your Google contact.
# Set up a service account as described in the README.
EE_ACCOUNT = 'gef-ldmp-server@gef-ld-toolbox.iam.gserviceaccount.com'

# The private key associated with your service account in JSON format.
EE_PRIVATE_KEY_FILE = 'D:/Gabriel/CI/TrendsEarth/Git/GeePython_Codes/te_key.json'

EE_CREDENTIALS = ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)

#ee.Initialize(EE_CREDENTIALS)
ee.Initialize()

In [ ]:
# Load region of interest
region = ee.Geometry(geojson)

# Filter region/country of interest
# countryName = ee.String('Guyana')
# country = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', countryName))

# Set year of interest
year = 2015

# Load the SDG15.3.1 layer in which pixel value = -1 represents degraded land
sdg1531 = ee.Image('users/geflanddegradation/global_ld_analysis/sdg1531_gpg_globe_2001_2015_modis').clip(country)
#print('SDG15.3.1: ', sdg1531)

# Filter pixels equal -1 == degraded land
ld = sdg1531.updateMask(sdg1531.eq(-1)).rename('DegLand')

# AGGREGATING DATASETS

# Load the WorldPop image with annual population count as bands
wp_imgCol = ee.Image("users/geflanddegradation/toolbox_datasets/worldpop_ppp_2000_2020_1km_global")
#print('WorldPop '+ year+': ', wp_imgCol.select('p'+year))

# Get WorldPop native projection before doing any computation
wpProj = ee.Image(wp_imgCol).projection();
#print('WorldPop Proj', wpProj);

# Get WorldPop native scale
wpRes = wpProj.nominalScale()
#print('WP Native Scale: ', wpRes)

# Filter WorldPop img to get band for the year of interest and clip it for the country of interest
wpYear = wp_imgCol
    .select('p'+year)             # select band
    .clip(country)                # clip band by the country outline
    .setDefaultProjection(wpProj) # assure filtered band has a set crs
#print('WorlPop Year: ', wpYear)

# Use reduceResolution to aggregate degraded land to the WorldPop cell size and grid
# Aggregate land degradation data pixels to 1km
ldAgg1km = ld.reduceResolution({
    'reducer': ee.Reducer.first(),
    'maxPixels': 1024
    }).reproject({
    'crs':wpProj // reproject to the worldpop scale == 1km
    })
#print(ldAgg1km)

# Summarize population count for the entire country at worldpop original scale
stats1km = wpYear.reduceRegion({
    'reducer': ee.Reducer.sum(),
    'geometry': country,
    'scale': wpRes,
    'maxPixels': 1e13,
    'tileScale': 16
})

stats1km = stats1km.get('p'+year)
#print('Total Country Pop at 1km: ', stats1km)

# Vectorize pixels representing degraded land
degLand = ldAgg1km.reduceToVectors({
    'geometry': country,
    'crs': wpProj,
    'scale': wpRes,
    'geometryType': 'polygon',
    'eightConnected': true,
    'labelProperty': 'DegLand',
    'bestEffort': true
}).union(1) # dissolve all polygons into a multipolygon
#print('DegLand FeatCol: ',degLand)


# Use reduceRegion() to get the a dictonary summarizing population exposed to degraded land
so2_l3_dic = wpYear.reduceRegion({
    'reducer': ee.Reducer.sum(),
    'geometry': degLand,
    'scale': wpRes,
    'maxPixels': 1e13,
    'tileScale': 16
})
#print('Pop exposed to LD: ', so2_l3_dic)

so2_l3_exposure = degLand.set({'Population_total': so2_l3_dic.getNumber('p'+year),
        'Percentage': so2_l3_dic.getNumber('p'+year).divide(stats1km).multiply(100),
        'Country': country,
        'Year': year
})
#print('SO2 L3 Exposure to Degraded Land '+countryName+'_'+year,so2_l3_exposure)

